# Exercícios do Capítulo 3 – Family Time

- Aluno: Matheus Vieira Honorio de Souza
- Matrícula: 525
- Curso: GES

## 1. Criação do Grafo

```cypher
// Criação dos Nós (Nodes)

// Pessoas
CREATE (:Pessoa:Engenheiro {nome: "João Silva", idade: 35, sexo: "masculino", salario: 8500});
CREATE (:Pessoa:Médica {nome: "Maria Oliveira", idade: 32, sexo: "feminino", especialidade: "Cardiologia"});
CREATE (:Pessoa:Professor {nome: "Carlos Souza", idade: 45, sexo: "masculino", disciplina: "Matemática"});
CREATE (:Pessoa:Estudante {nome: "Ana Costa", idade: 20, sexo: "feminino", curso: "Medicina"});

// Pets
CREATE (:Pet:Cachorro {nome: "Rex", raca: "Labrador", idade: 5, peso: 28.5});
CREATE (:Pet:Gato {nome: "Mingau", raca: "Siamês", idade: 3, cor: "branco"});

// Instituições
CREATE (:Empresa:Tecnologia {nome: "Empresa X", fundacao: 2005, funcionarios: 150, setor: "TI"});
CREATE (:Instituicao:Educacao {nome: "Universidade Y", fundacao: 1960, cursos: 25, publica: true});

// Produtos
CREATE (:Produto:Livro {titulo: "Dom Casmurro", autor: "Machado de Assis", ano: 1899, preco: 29.90});
CREATE (:Produto:Eletronico {modelo: "Galaxy S20", marca: "Samsung", ano: 2020, preco: 3999.90});

// Criação dos Relacionamentos (Relationships)

// Relacionamentos PAI_DE (com propriedade)
MATCH (p1:Pessoa {nome: "João Silva"}), (p2:Pessoa {nome: "Ana Costa"})
CREATE (p1)-[:PAI_DE {desde: "2010-05-15"}]->(p2);

MATCH (p1:Pessoa {nome: "Carlos Souza"}), (p2:Pessoa {nome: "Ana Costa"})
CREATE (p1)-[:PAI_DE {desde: "2003-07-22"}]->(p2);

// Relacionamentos TRABALHA_EM
MATCH (p:Pessoa {nome: "João Silva"}), (e:Empresa {nome: "Empresa X"})
CREATE (p)-[:TRABALHA_EM {cargo: "Engenheiro Chefe"}]->(e);

MATCH (p:Pessoa {nome: "Maria Oliveira"}), (i:Instituicao {nome: "Universidade Y"})
CREATE (p)-[:TRABALHA_EM {cargo: "Cardiologista"}]->(i);

// Relacionamentos DONO_DE
MATCH (p:Pessoa {nome: "João Silva"}), (pet:Pet {nome: "Rex"})
CREATE (p)-[:DONO_DE]->(pet);

MATCH (p:Pessoa {nome: "Ana Costa"}), (pet:Pet {nome: "Mingau"})
CREATE (p)-[:DONO_DE]->(pet);

// Relacionamento ESTUDA_EM
MATCH (p:Pessoa {nome: "Ana Costa"}), (i:Instituicao {nome: "Universidade Y"})
CREATE (p)-[:ESTUDA_EM {matricula: "2023001"}]->(i);

// Relacionamentos COMPROU (com propriedades)
MATCH (p:Pessoa {nome: "Maria Oliveira"}), (prod:Produto {titulo: "Dom Casmurro"})
CREATE (p)-[:COMPROU {data: "2023-01-10", valor: 35.90}]->(prod);

MATCH (p:Pessoa {nome: "Carlos Souza"}), (prod:Produto {modelo: "Galaxy S20"})
CREATE (p)-[:COMPROU {data: "2023-02-15", valor: 3500.00}]->(prod);

// Consulta para verificar o grafo criado
MATCH (n) RETURN n;
```

In [1]:
from neo4j import GraphDatabase

class Neo4jClient:
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self._driver.close()
    
    def run_query(self, query, parameters=None):
        with self._driver.session() as session:
            result = session.run(query, parameters)
            return [record for record in result]
    
    def get_engineers_in_family(self):
        query = """
        MATCH (p:Pessoa:Engenheiro)
        RETURN p.nome AS nome, p.idade AS idade, p.salario AS salario
        """
        return self.run_query(query)
    
    def get_children_of(self, parent_name):
        query = """
        MATCH (parent:Pessoa {nome: $parent_name})-[:PAI_DE]->(child:Pessoa)
        RETURN child.nome AS nome, child.idade AS idade, child.curso AS curso
        """
        return self.run_query(query, {'parent_name': parent_name})
    
    def get_pets_of(self, owner_name):
        query = """
        MATCH (owner:Pessoa {nome: $owner_name})-[:DONO_DE]->(pet:Pet)
        RETURN pet.nome AS nome, pet.raca AS raca, pet.idade AS idade
        """
        return self.run_query(query, {'owner_name': owner_name})
    
    def get_work_info(self, person_name):
        query = """
        MATCH (p:Pessoa {nome: $person_name})-[r:TRABALHA_EM]->(e)
        RETURN e.nome AS empresa, r.cargo AS cargo
        """
        return self.run_query(query, {'person_name': person_name})

def main():
    # Configuração da conexão - altere para suas credenciais
    uri = "bolt://localhost:7687"
    user = "neo4j"
    password = "matheus1032"  # Substitua pela sua senha
    
    client = Neo4jClient(uri, user, password)
    
    try:
        print("\n1. Quem da família é Engenheiro?")
        engineers = client.get_engineers_in_family()
        for eng in engineers:
            print(f"- {eng['nome']} (Idade: {eng['idade']}, Salário: R${eng['salario']:.2f}")
        
        print("\n2. João Silva é pai de quem?")
        children = client.get_children_of("João Silva")
        for child in children:
            print(f"- {child['nome']} (Idade: {child['idade']}, Curso: {child['curso']})")
        
        print("\n3. Quais são os pets de Ana Costa?")
        pets = client.get_pets_of("Ana Costa")
        for pet in pets:
            print(f"- {pet['nome']} (Raça: {pet['raca']}, Idade: {pet['idade']} anos)")
        
        print("\n4. Onde Maria Oliveira trabalha e qual seu cargo?")
        work_info = client.get_work_info("Maria Oliveira")
        for info in work_info:
            print(f"- Empresa: {info['empresa']}, Cargo: {info['cargo']}")
    
    finally:
        client.close()

if __name__ == "__main__":
    main()


1. Quem da família é Engenheiro?
- João Silva (Idade: 35, Salário: R$8500.00

2. João Silva é pai de quem?
- Ana Costa (Idade: 20, Curso: Medicina)

3. Quais são os pets de Ana Costa?
- Mingau (Raça: Siamês, Idade: 3 anos)

4. Onde Maria Oliveira trabalha e qual seu cargo?
- Empresa: Universidade Y, Cargo: Cardiologista
